In [4]:
# Load rosters and the tables
# Then print name and surname for team ID
import pandas as pd
import appearance
import generaldata
import ratings

season = 1996
season_text = f"{season-1}-{str(season)[2:]}"
yeardiff = 2013 - season

# if game is 2005 then game value should be 9
is2005 = True
if is2005:
    generic_faces = pd.read_csv('genericfaces05_v01.csv')
    game = 9
else:
    generic_faces = pd.read_csv('genericfaces06_v01.csv')
    game = 8
# Get the necessary tables. Used to filter 2k players table for only active teams.
nickname_table = pd.read_csv('nicknames_v01.csv', dtype={'ID': int, 'NICKNAME': str})
compare = pd.read_csv('compare_v01.csv')
colleges = pd.read_csv('colleges_v01.csv')

# Read player and team tables of 2K roster. FA players are filtered separately as they will be moved to Free Agents (team 50)
players_2k = pd.read_csv(f'{season_text}//Players.csv', encoding="utf-16")
# teams_2k = pd.read_csv(f'{season_text}//Teams.csv', encoding="utf-16", delimiter='\t')
teams_2k = pd.read_csv(f'{season_text}//Teams.csv', encoding="utf-16")
players_fa = players_2k[players_2k["IsFA"] == 1]

# Gets the team table. Used to filter 2k players table for only active teams.
teamIDs = pd.read_csv('teamIDmatch.csv')
# Run the filter with teamIDs. For each 2KID check IS85 (which means if team is active in 1984-85 season) is true.
# For each ID in 1st row, if IS85 is true then append to teamList
teamList = teamIDs.loc[teamIDs[f"IS{season % 100}"] != "FALSE", "2KID"].tolist()

players_withteam = players_2k[players_2k["TeamID1"].isin(teamList)]
players_nba = pd.concat([players_withteam, players_fa])

In [5]:
# Create rows for database

players_live = []
appearance_live = []

spareID = 3000

for record in players_nba.iterrows():
    pl_record = record[1]
    # Create new tuple with DELETED value
    playerlive = [False]
    appearlive = [False]
    # Last and first names NAME, FNAME
    name, fname = pl_record["Last_Name"], pl_record["First_Name"]
    playerlive.append(name[:13])
    playerlive.append(fname[:12])
    playerlive.append(generaldata.check_nickname(record, nickname_table))
    # ID. Now this is tricky. I need to have birth date of player
    birthdate = ((pl_record["BirthYear"] - game) * 10000) + (pl_record["BirthMonth"] * 100) + pl_record["BirthDay"]
    # Actual birthdate of the player
    actual_birthdate = birthdate - ((yeardiff - game) * 10000)
    # Checks if player is available at comparison table.
    comparedata = generaldata.check_playerID(compare, name, fname, actual_birthdate)
    if type(comparedata) != list:
        # append ID for the player. use values[0]
        playerID = comparedata["PLAYERID"].values[0]
        playerlive.append(comparedata["PLAYERID"].values[0])
        appearlive.append(comparedata["PLAYERID"].values[0])
    else:
        # Copy spareID to playerID that would be used for the player.
        playerID = spareID
        # Increment spareID to avoid duplicate ID uses immediately.
        spareID += 1
        playerlive.append(playerID)
        appearlive.append(playerID)
    appearlive.append(-1)
    # Height and weight
    height = round(pl_record['Height']/2.54)
    weight = round(pl_record["Weight"])
    skintone_2k = pl_record["SkinTone"]
    skintone = appearance.set_skintone(skintone_2k)
    eyewear = 0
    if type(comparedata) != list:
        # as there are empty values they are taken as float. if it is equal or larger than 0.0 then it means it has CF. weird but it is how it works.
        if comparedata["SKINTONE"].values[0] >= 0.0:
            skintone = int(comparedata["SKINTONE"].values[0])
            id7 = comparedata["ID7"].values[0]
            eyewear = int(comparedata["EYEWEAR"].values[0])
        else:
            # get id7 from genericfaces
            id7 = appearance.select_random_pkg(generic_faces, skintone)
    else:
        # get id7 from genericfaces
        id7 = appearance.select_random_pkg(generic_faces, skintone)
    # Set positions
    if is2005:
        appearlive.append(id7)
    appearlive.append(height)
    appearlive.append(weight)
    appearlive.append(skintone)
    # Player jersey number
    number = pl_record['Number']
    if number == 100:
        playerlive.append(-1)
    else:
        playerlive.append(number)
    pos, secondpos = pl_record['Pos'], pl_record['SecondPos']
    positions = generaldata.set_position(pos, secondpos)
    playerlive += positions
    # Set years of experience
    playerlive.append(pl_record['YearsPro'] - 1)
    # Set college
    college = generaldata.set_college(colleges, pl_record['CollegeID'], is2005)
    playerlive.append(college)
    # Set hand
    playerlive.append(-(pl_record['Hand']) + 1)
    # Set shoe brand and shoes... for shoes temporarily they will be 0, 0
    shoebrand = generaldata.set_shoe_brand(pl_record['GShsBrLck'])
    playerlive.append(shoebrand)
    playerlive += [0, 0]
    # Look up for a combination of name, surname and birthdate. Then check USE value. If it is true then some values will be used from the related row.
    attributes = ratings.calc_attributes(record, is2005)
    playerlive += attributes
    # Facial ctrl, freethrow, audio, celebpack
    facialctrl = max(pl_record['Personality'] - 1, 0)
    playerlive += [facialctrl, 0, None, facialctrl]
    playerlive.append(ratings.calc_threeptsty(height, ratings.convert_rating(pl_record['SSht3PT'], is2005)))
    # Gets team value, original team value and roster position
    if pl_record["IsFA"] == 0:
        team = teamIDs.loc[teamIDs['2KID'] == pl_record['TeamID1'], 'EAID'].values[0]
        origteam = team
        # Executed for All-Star teams
        if team == 30 or team == 31:
            # Find the first instance of player and grab team ID of that one.
            name, fname
            # Find the first row matching the query
            row = players_nba[(players_nba['First_Name'] == fname) & (players_nba['Last_Name'] == name)].iloc[0]
            # Get the value of 'TeamID1' from the first row
            team_id = row['TeamID1']
            origteam = teamIDs.loc[teamIDs['2KID'] == team_id, 'EAID'].values[0]
        rosterpos = generaldata.set_roster_position(teams_2k, pl_record['TeamID1'], pl_record['ID'])
        playerlive += [origteam, team]
        playerlive.append(rosterpos)
    else:
        team = 50
        playerlive += [team, team]
        playerlive.append(0)
    # Now find the column where player ID is stored at the team row. this determines roster pos
    # Find the row that has the value at ID column
    playerlive.append(birthdate)
    # for now birthtown is Not Set for everyone
    playerlive.append(2024)
    # draft picks
    draftteam = pl_record['DraftedBy']
    draftyear = pl_record['DraftYear']
    draftdata = generaldata.draftdata(record, teamIDs, draftteam)
    playerlive += draftdata
    # scratches
    playerlive += [0,0,1]
    # dunkpack
    dunkpack = ratings.calc_dunkpack_06(height, ratings.convert_rating(pl_record['SDunk'], is2005), weight)
    destiny = ratings.calc_destiny((pl_record["BirthYear"] - game), is2005)
    playerlive.append(dunkpack)
    playerlive.append(destiny)
    # injuryID,date,recovrdate
    playerlive += [0, None, None]
    if is2005:
        # isplayable,iscreated,facepatch,isgenerate,ishistoric,tradeable
        playerlive += (True, False, False, False, False, None)
        # playerPKG, INSEASON, INPLAYOFF, rookdata, gradclass
        playerlive.append(id7)
        playerlive += [False, False, 0, 0, 0, 0, 0]
    else:
        # isplayable,iscreated,isgenerate,ishistoric,tradeable
        playerlive += (True, False, False, False, None)
        # playerPKG, INSEASON, INPLAYOFF, rookdata
        playerlive.append(id7)
        playerlive += [False, False, 0, 0, 0, 0]
    # CASH_YEAR	CASH_TOTAL	CASH_RAISE	YEARS_LONG	YEARS_LEFT	CASH_TEAM	
    if pl_record["IsFA"] == 0:    
        playerlive.append(int(pl_record["CYear1"])/10000)
        playerlive.append(0)
        playerlive.append(int(max((pl_record["CYear2"] - pl_record["CYear1"]), 0))/10000)
        playerlive.append(pl_record["CClrYears"])
        playerlive.append((1 if pl_record["CYear1"] != 0 else 0)+
                        (1 if pl_record["CYear2"] != 0 else 0)+
                        (1 if pl_record["CYear3"] != 0 else 0)+
                        (1 if pl_record["CYear4"] != 0 else 0)+
                        (1 if pl_record["CYear5"] != 0 else 0)+
                        (1 if pl_record["CYear6"] != 0 else 0))
        playerlive.append(team)
    else:
        playerlive += [0,0,0,0,0,team]
    # PREF_LEN	MIN_TARGET	LOWBALLED	ISMINIMUM
    playerlive += [0, 0, 0, False]
    if is2005:
        # rest of season data etc for now no plans
        playerlive += [0] * 76
    else:
        playerlive += [0, 0]
        # superstar, off, def - no plans for this either
        playerlive += [0] * 3
        # rest of season data etc for now no plans
        playerlive += [0] * 68
        # learning, workethic. bird, cashresign, speechID, specmap, isedited, isorigss, validsstar, dfltteam
        playerlive.append(ratings.convert_rating(pl_record['SPOT'], is2005))
        playerlive += [2, 0, 0, playerID, 3, False, False, 0, team]
    # add record to players
    players_live.append(tuple(playerlive))
    # Now, appearance
    # BODYTYPE MUSCLETEXT FACE HAIRTYPE
    appearlive.append(appearance.set_bodytype(pl_record['Bodytype']))
    appearlive.append(pl_record['Muscles'])
    appearlive.append(0)
    # For hairtype in near future, if player has braids then 2, if we have hair model then 1. But in upcoming updates
    appearlive.append(0)
    # EYECOLOR, EYEWEAR, HEADWEAR
    appearlive.append(appearance.set_eyecolor(pl_record['EyeColor']))
    appearlive.append(eyewear)
    appearlive.append(pl_record['GHeadband'])
    appearlive += [16119285, 8984857, 3557269]
    # shoes, logo, sock, sockcolor
    appearlive += [0, 0, 1]
    appearlive.append(appearance.set_sock(pl_record['GSockLngh']))
    appearlive += [16119285, 8984857, 3557269]
    # neck tattoo, fingers
    appearlive += [0, 0]
    appearlive.append(appearance.set_sock(pl_record['GLeftFngr']))
    appearlive.append(16119285)
    appearlive.append(appearance.set_sock(pl_record['GRghtFngr']))
    appearlive.append(16119285)
    # wrist
    appearlive.append(appearance.set_wrist(pl_record['GLeftWrst']))
    appearlive += [None, None]
    appearlive.append(appearance.set_wrist(pl_record['GRghtWrst']))
    appearlive += [None, None]
    # arm accessories
    l_arm2k, l_elb2k = pl_record['GLeftArm'], pl_record['GLeftElb']
    r_arm2k, r_elb2k = pl_record['GRghtArm'], pl_record['GRghtElb']
    l_bic, l_elb, l_for = appearance.set_arm(l_arm2k, l_elb2k)
    r_bic, r_elb, r_for = appearance.set_arm(r_arm2k, r_elb2k)
    appearlive += [l_for, None, r_for, None, l_elb, None, r_elb, None, l_bic, None, r_bic, None, 0, 0, 0, 0]
    # short shorts, pressure shorts
    if season > 1990:
        appearlive.append(False)
    else:
        appearlive.append(True)
    if pl_record['GPresShrt'] != 0:
        appearlive += [1, None]
    else:
        appearlive += [0, None]
    # leg accessories
    # LKNEEPAD	LKNEECOL	LKNEECOL2	RKNEEPAD	RKNEECOL	RKNEECOL2	LLOWLEGTAT	RLOWLEGTAT	LLOWLEG	LLOWLEGCOL	LLOWLEGCO2	RLOWLEG	RLOWLEGCOL	RLOWLEGCO2
    l_knee2k, l_leg2k = pl_record['GLeftKnee'], pl_record['GLeftLeg']
    r_knee2k, r_leg2k = pl_record['GRghtKnee'], pl_record['GRghtLeg']   
    l_knee, l_leg = appearance.set_legs(l_knee2k, l_leg2k)
    r_knee, r_leg = appearance.set_legs(r_knee2k, r_leg2k)
    appearlive += [l_knee, None, None, r_knee, None, None, None, None, l_leg, None, None, r_leg, None, None]
    # age
    if not is2005:
        appearlive.append(None)
    # add record to appearance
    appearance_live.append(tuple(appearlive))
    # learning
    # Overall will be the next update


In [6]:
# Import rows to DBF

from dbfread import DBF
import dbf
import fieldformat

if is2005:
    savepath = '2005save'
    outputpath = "2005output"
else:
    savepath = '2006save'
    outputpath = "2006output"
player = f'{savepath}/players.dbf'
new_player = f'{outputpath}/players.dbf'
appearance = f'{savepath}/appearance.dbf'
new_appearance = f'{outputpath}/appearance.dbf'

player_records = []
player_IDs = []
appearance_records = []

# Read records at players
for record in DBF(player):
    if record['TEAM'] > 31 and record['TEAM'] < 50:
        player_records.append(tuple(record.values()))
        player_IDs.append(record['PLAYERID'])

# Read records at appearance
for record in DBF(appearance):
    if record['APPEARID'] in player_IDs:
        appearance_records.append(tuple(record.values()))

# Create new table with fields received fromdbfread
def createDbf(new_filename, filename, filtered_records, datafrom2k):
    table = dbf.Table(
            filename=new_filename,
            field_specs=fieldformat.field_formatter(filename),
            on_disk=True,
            )

    # Open, append and save table
    table.open(dbf.READ_WRITE)

    for record in datafrom2k:
        try:
            table.append(record)
        except:
            print(record)

    # Write records
    for datum in filtered_records:
        table.append(datum)

    # Conclude
    table.close()

createDbf(new_player, player, player_records, players_live)
createDbf(new_appearance, appearance, appearance_records, appearance_live)



DataOverflowError: field 'DRAFTEDBY': ('tried to store 5 bytes in 4 byte field',)